读取csv文件数据

In [7]:
import pandas as pd
import os

# 指定包含CSV文件的目录
data_dir = 'datacre'

# 列出目录下所有的CSV文件
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

# 初始化一个空的DataFrame
all_data = pd.DataFrame()

# 读取每个CSV文件并合并
for file in csv_files:
    file_path = os.path.join(data_dir, file)
    df = pd.read_csv(file_path)
    all_data = pd.concat([all_data, df], ignore_index=True)

# 显示合并后的数据框的前几行
print(all_data.head())


      videoid                                         contentUrl     duration  \
0    21179416  https://ak.picdn.net/shutterstock/videos/21179...  PT00H00M11S   
1     5629184  https://ak.picdn.net/shutterstock/videos/56291...  PT00H00M29S   
2  1063125190  https://ak.picdn.net/shutterstock/videos/10631...  PT00H00M07S   
3  1039695998  https://ak.picdn.net/shutterstock/videos/10396...  PT00H00M08S   
4     9607838  https://ak.picdn.net/shutterstock/videos/96078...  PT00H00M32S   

        page_dir                                               name  
0  006001_006050                          Aerial shot winter forest  
1  071501_071550  Senior couple looking through binoculars on sa...  
2  133701_133750  A beautiful cookie with oranges lies on a gree...  
3  011901_011950  Japanese highrise office skyscrapers tokyo square  
4  121551_121600  Zrenjanin,serbia march 21 2015: fans watching ...  


查找某类元素

In [8]:
# 查找'name'列中包含'tree'的行
contains_video = all_data[all_data['name'].str.contains('tree', na=False)]

# 获取DataFrame的行数和列数
rows, cols = contains_video.shape

# 打印行数
print(f"总行数: {rows}")

# 显示结果
print(contains_video.head())

总行数: 2836
       videoid                                         contentUrl  \
12  1047392872  https://ak.picdn.net/shutterstock/videos/10473...   
35  1018923682  https://ak.picdn.net/shutterstock/videos/10189...   
48  1027734908  https://ak.picdn.net/shutterstock/videos/10277...   
53    22430842  https://ak.picdn.net/shutterstock/videos/22430...   
69     8309845  https://ak.picdn.net/shutterstock/videos/83098...   

       duration       page_dir  \
12  PT00H00M15S  100401_100450   
35  PT00H00M11S  028901_028950   
48  PT00H00M34S  136101_136150   
53  PT00H00M20S  141851_141900   
69  PT00H00M08S  118551_118600   

                                                 name  
12  Red squirrel bushy tail sitting watching on tr...  
35       Blurred city street view. city traffic light  
48  Aerial drone video of a forest surrounding far...  
53      Frosty spruce trees in snow isolated on white  
69  Mexico city - circa 1973: food stand in the st...  


数据处理

In [9]:
columns_to_keep = ['videoid', 'contentUrl', 'name']
df_filtered = contains_video[columns_to_keep]

# 获取DataFrame的行数和列数
rows, cols = df_filtered.shape

# 打印行数
print(f"总行数: {rows}")

# 显示结果
print(df_filtered.head())

总行数: 2836
       videoid                                         contentUrl  \
12  1047392872  https://ak.picdn.net/shutterstock/videos/10473...   
35  1018923682  https://ak.picdn.net/shutterstock/videos/10189...   
48  1027734908  https://ak.picdn.net/shutterstock/videos/10277...   
53    22430842  https://ak.picdn.net/shutterstock/videos/22430...   
69     8309845  https://ak.picdn.net/shutterstock/videos/83098...   

                                                 name  
12  Red squirrel bushy tail sitting watching on tr...  
35       Blurred city street view. city traffic light  
48  Aerial drone video of a forest surrounding far...  
53      Frosty spruce trees in snow isolated on white  
69  Mexico city - circa 1973: food stand in the st...  


In [ ]:
# [{"value": 2, "label": "a bear", "logit": 0.31, "box": [110.57392883300781, 1.652069091796875, 593.0380859375, 331.99102783203125]}]

数据输出

In [10]:
df_filtered.to_csv('tree.csv', index=False)

In [12]:
!pip install pandas opencv-python torchvision

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   - -------------------------------------- 1.8/38.8 MB 14.4 MB/s eta 0:00:03
   -------- ------------------------------- 8.7/38.8 MB 29.9 MB/s eta 0:00:02
   ----------------- ---------------------- 16.5/38.8 MB 30.6 MB/s eta 0:00:01
   -------------------------- ------------- 25.4/38.8 MB 34.3 MB/s eta 0:00:01
   --------------------------------- ------ 32.2/38.8 MB 34.1 MB/s eta 0:00:01
   ---------------------------------- ----- 33.6/38.8 MB 34.4 MB/s eta 0:00:01
   ---------------------------------------- 38.8/38.8 MB 28.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 41.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
    --------------------------------------- 3.1/203.0 MB 90.1 MB/s eta 0:00:03
   -- ------------------------------------- 11.5/203.0 MB 42.4 MB/s eta 0:00:05
   

In [13]:
import requests
import cv2
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

def download_video(url, filename):
    """从网址下载视频文件"""
    response = requests.get(url, stream=True)
    with open(filename, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)

def extract_frames(video_path, frame_interval=10):
    """从视频中提取帧"""
    frames = []
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_interval == 0:
            frames.append(frame)
        frame_count += 1
    cap.release()
    return frames

def detect_objects(frames, model, device):
    """使用预训练模型进行对象检测"""
    results = []
    model.eval()
    with torch.no_grad():
        for frame in frames:
            img_tensor = F.to_tensor(frame).unsqueeze(0).to(device)
            predictions = model(img_tensor)
            for element in range(len(predictions[0]["labels"])):
                logit = predictions[0]["scores"][element].item()
                box = predictions[0]["boxes"][element].tolist()
                label = predictions[0]["labels"][element].item()
                results.append({
                    "value": label,
                    "label": f"Class {label}",
                    "logit": logit,
                    "box": box
                })
    return results

# 下载视频
video_url = 'https://ak.picdn.net/shutterstock/videos/1047392872/preview/stock-footage-red-squirrel-bushy-tail-sitting-watching-on-tree-branch-slow-motion.mp4'
video_filename = 'downloaded_video.mp4'
download_video(video_url, video_filename)

# 提取帧
frames = extract_frames(video_filename, frame_interval=10)

# 加载预训练模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = fasterrcnn_resnet50_fpn(pretrained=True).to(device)

# 进行对象检测
detection_results = detect_objects(frames, model, device)

# 打印结果
for result in detection_results:
    print(result)

c:\Users\86187\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\86187\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\86187/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:06<00:00, 26.4MB/s] 


{'value': 16, 'label': 'Class 16', 'logit': 0.28881996870040894, 'box': [228.59423828125, 45.2222900390625, 447.9231262207031, 257.8106994628906]}
{'value': 23, 'label': 'Class 23', 'logit': 0.2811362147331238, 'box': [255.52621459960938, 63.5997428894043, 458.1296691894531, 265.6768798828125]}
{'value': 75, 'label': 'Class 75', 'logit': 0.16888853907585144, 'box': [345.7100524902344, 190.09080505371094, 367.6836242675781, 210.6385040283203]}
{'value': 22, 'label': 'Class 22', 'logit': 0.13139046728610992, 'box': [227.91854858398438, 32.87535858154297, 462.4887390136719, 268.4133605957031]}
{'value': 21, 'label': 'Class 21', 'logit': 0.12444982677698135, 'box': [248.2668914794922, 56.052860260009766, 461.99029541015625, 267.069580078125]}
{'value': 18, 'label': 'Class 18', 'logit': 0.11484447866678238, 'box': [282.7768859863281, 62.88679885864258, 460.0258483886719, 237.15223693847656]}
{'value': 19, 'label': 'Class 19', 'logit': 0.09730841219425201, 'box': [186.0825653076172, 27.84638